## **RAG 기본 구조 이해하기**

### **1. 사전작업(Pre-processing) - 1~4 단계**

<img src='https://ifh.cc/g/VK9RJ9.png' width = "800">
<img src='https://velog.velcdn.com/images/jjlee6496/post/f4443dd4-cdca-4e4c-83e6-8ec515c21db5/image.png' width = "800">


사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행

- 1단계 : 도큐먼트 로드 (Document Loader): 외부 데이터 소스에서 필요한 문서를 로드하고 초기 처리. 
    - 이것은 마치 책을 여러 권 챙겨 도서관에서 공부하는 것과 비슷. 학생이 공부하기 전에 필요한 책들을 책장에서 골라오는 과정.

- 2단계 : 텍스트 분할 (Text Splitter): 로드된 문서를 처리 가능한 작은 단위로 분할. 큰 책을 챕터별로 나누는 것과 유사.

- 3단계 : 임베딩 (Embedding): 각 문서 또는 문서의 일부를 벡터 형태로 변환하여, 문서의 의미를 수치화. 이는 책의 내용을 요약하여 핵심 키워드로 표현하는 것과 비슷.

- 4단계 : 벡터스토어(Vector Store) 저장: 임베딩된 벡터들을 데이터베이스에 저장. 이는 요약된 키워드를 색인화하여 나중에 빠르게 찾을 수 있도록 하는 과정.


### **2. RAG 수행(RunTime) - 5~8 단계**

<img src='https://ifh.cc/g/plorDw.png' width = "800">

<img src='https://velog.velcdn.com/images/jjlee6496/post/15b30fe5-1015-47b4-b837-d692f9a101fd/image.png' width = "800">


- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 Retriever 정의.
    - Retriever는 검색 알고리즘
    - Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됨. 
    - 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있음.
- 7단계 LLM: 모델을 정의.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성.

In [1]:
# RAG에 필요한 모듈 import 
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import TavilySearchAPIRetriever

### **Load Documents**

In [5]:
loader = PyMuPDFLoader(r'files/test.pdf')
docs = loader.load()
print(f'{len(docs)} pages docs')

2 pages docs


### **Text Split**

In [6]:
# Split text into characters
splitter = RecursiveCharacterTextSplitter(chunk_size = 415, chunk_overlap = 0)
split_docs = splitter.split_documents(docs)    # chunk_size : 몇 단어씩 자를 것인지, chunk_overlap : 앞 페이지와 겹치는 단어 수

print(split_docs[1].page_content)

이영희 
 2. 
(Younghee Lee)
직책
마케팅 매니저 
- 
: 
(Marketing Manager)
부서
마케팅 
- 
: 
(Marketing)
이메일
- 
: yh.lee@company.com
전화번호
- 
: 010-2345-6789
입사일
년 월 
일
- 
: 2019
7
22
연봉
만 원
- 
: 8,000
특징
- 
:
창의적인 아이디어를 기반으로 한 마케팅 전략 수립 능력이 뛰어남
  - 
데이터 분석을 통해 마케팅 성과를 측정하고 최적화하는 능력이 있음
  - 
대인관계가 뛰어나고 협력업체 및 고객과의 관계를 잘 유지함
  - 
담당 업무
- 
:
디지털 마케팅 캠페인 기획 및 실행
  - 
브랜드 전략 수립 및 시장 조사
  - 
광고 및 프로모션 운영
성과 분석
  - 
, 
박민준 
 3.


### **Embedding**

In [7]:
# 기본 모델 : text-embedding-ada-002
embeded = OpenAIEmbeddings(model = 'text-embedding-3-small')

### **VectorDB 생성**

In [ ]:
vector_store = FAISS.from_documents(documents=split_docs, embedding=embeded)

In [9]:
import tiktoken

model = 'text-embedding-3-small'

def embeded_token_cost(model, docs):
    # 토큰 비용 계산기
    token_prices = {
        "text-embedding-3-small": 0.02 / 1_000_000,
        'text_embedding-3-large' : 0.13 / 1_000_000,
        'text-embedding-ada-002' : 0.1 / 1_000_000,
    }

    input_token_cost = token_prices[model]
    input_text = ''.join([i.page_content for i in docs])


    in_out = {'input_text': [input_text, input_token_cost]}
    krw = 1451.68

    for k, v in in_out.items():

        encoding = tiktoken.encoding_for_model(model)
        tokens = encoding.encode(v[0])

        num_tokens = len(tokens)
        print(f"{k} tokens: {num_tokens}")
        print(v[1])
        print(f"{k} cost: {num_tokens * v[1]:.5f}$")
        print(f"{k} cost: {num_tokens * v[1]* krw:.2f}원\n")
    
embeded_token_cost(model, docs)

input_text tokens: 1055
2e-08
input_text cost: 0.00002$
input_text cost: 0.03원



### **Retriever 생성**

[Tavily 참고](https://velog.io/@woody_ahn/Tavily-LLM%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%B5%9C%EC%A0%81%ED%99%94%EB%90%9C-%EA%B2%80%EC%83%89-API)

웹 검색 Retriver

In [8]:
import os 

os.environ['TAVILY_API_KEY'] = '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'

In [17]:
retriver = TavilySearchAPIRetriever(k=3)
result = retriver.invoke('에스티이노베이션이라는 회사의 2023년도 매출액은 얼마인가요?')
result

[Document(metadata={'title': '(주)에스티이노베이션 기업정보 2025년 | NICEbizinfo (NICE기업정보)', 'source': 'https://www.nicebizinfo.com/ep/EP0100M002GE.nice?kiscode=JE2605', 'score': 0.4889428, 'images': []}, page_content='(주)에스티이노베이션 기업정보, 신용정보, 등급정보, 재무정보, 연봉정보, 특허정보, 인사, 부고 모두 NICEbizinfo에서 확인하세요. ... 2023 2022 2021 ... * 최근 매출액은 외감법인 및 상장사에 한해서만 공개됩니다.'),
 Document(metadata={'title': '2024년 국내 제약회사 순위 매출액 기준 Top 30 : 네이버 블로그', 'source': 'https://m.blog.naver.com/csss3324/223539361402', 'score': 0.37353352, 'images': []}, page_content='2024년 국내 제약회사 순위 매출액 기준 Top 30 : 네이버 블로그 2024년 국내 제약회사 순위 매출액 기준 Top 30 2024년 국내 제약회사 순위 #국내제약회사순위 #제약회사순위 2024년 국내 제약회사 순위가 발표되었습니다. 이 순위는 2023년 매출액을 기준으로 한 것으로, 매출액: 8,596억 2,558만 원 (2023년 기준) 매출액: 8,289억 823만 원 (2023년 기준) 매출액: 7,485억 2,670만 원 (2023년 기준) 매출액: 7,309억 9,359만 원 (2023년 기준) 매출액: 7,263억 9,375만 원 (2023년 기준) 매출액: 6,639억 8,036만 원 (2023년 기준) 매출액: 6,310억 2,643만 원 (2023년 기준) 매출액: 6,007억 5,784만 원 (2023년 기준) 2024년 국내 제약회사 순위는 각 회사의 경제적 성과와 시장에서의 이 정보는 각 기업의 규모와 성

In [10]:
retriver = vector_store.as_retriever()

# 검색기가 어떤 문서를 참고하는지 확인
retriver.invoke('김철수씨의 업무는 어떤게 있습니까?')

[Document(id='0926f6b2-f57d-4c24-a782-c934cc396f70', metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2025-03-11T23:37:13+09:00', 'source': 'files/test.pdf', 'file_path': 'files/test.pdf', 'total_pages': 2, 'format': 'PDF 1.7', 'title': 'HWP Document', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-03-11T23:37:13+09:00', 'trapped': '', 'modDate': "D:20250311233713+09'00'", 'creationDate': "D:20250311233713+09'00'", 'page': 0}, page_content='회사 사원 정보\n김철수 \n 1. \n(Chulsoo Kim)\n직책\n소프트웨어 엔지니어 \n- \n: \n(Software Engineer)\n부서\n엔지니어링 \n- \n: \n(Engineering)\n이메일\n- \n: cs.kim@company.com\n전화번호\n- \n: 010-1234-5678\n입사일\n년 월 \n일\n- \n: 2018\n3\n15\n연봉\n만 원\n- \n: 7,000\n특징\n- \n:\n문제 해결 능력이 뛰어나며\n복잡한 알고리즘을 설계하는 것을 즐김\n  - \n, \n새로운 기술 습득이 빠르며\n최신 \n트렌드에 대한 관심이 높음\n  - \n, \nIT \n팀원들과의 협업을 중요하게 생각하는 스타일\n  - \n담당 업무\n- \n:\n사내 웹 애플리케이션 개발 및 유지보수\n  - \n신규 프로젝트의 백엔드 시스템 설계\n  - \n코드 리뷰 및 신입 개발자 멘토링\n  -'),
 Document(id='61aff751-2130-4ccf-8705-

### **프롬프트 생성(Create Prompt)**


In [10]:
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

### **언어모델 생성(Chain)**

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatGroq(model = 'gemma2-9b-it')

chain = (
    {'context' : retriver, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### **출력**

In [14]:
response = chain.stream('''
                        에스티이노베이션이라는 회사의 2023년도 매출액은 얼마인가요?
                        ''')

for c in response:
    print(c, end='', flush=True)

제공된 텍스트에는 에스티이노베이션의 2023년 매출액에 대한 정보가 없습니다. 


